In [1]:
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import psycopg2
import psycopg2.extras

import pandas as pd
import numpy as np

import json
import logging
import re
import time
# from sql_queries import *

In [2]:
# URL for all laptops on B&H website
theurl = 'https://www.bhphotovideo.com/c/buy/laptops/ci/18818/N/4110474292/pn/'

In [3]:
# laptop table insert query
laptop_table_insert = """INSERT INTO dimlaptop (name, sku, url)
                            SELECT name, sku, url
                            FROM staging_laptop2
                            ON CONFLICT (sku)
                            DO UPDATE SET name = excluded.name,
                                          url = excluded.url;
                      """

staging_table_insert = """INSERT INTO staging_laptop2 (name, time, brand, sku, price, reg_price, money_saved, url, availability, review_num)
                                VALUES ( %(name)s,
                                         %(time)s,
                                         %(brand)s,
                                         %(sku)s,
                                         %(price)s,
                                         %(reg_price)s,
                                         %(money_saved)s,
                                         %(url)s,
                                         %(availability)s,
                                         %(review_num)s
                                        )
                                ON CONFLICT (sku)
                                DO UPDATE SET name = excluded.name,
                                              time = excluded.time,
                                              brand = excluded.brand,
                                              sku = excluded.sku,
                                              price = excluded.price,
                                              reg_price = excluded.reg_price,
                                              money_saved = excluded.money_saved,
                                              url = excluded.url,
                                              availability = excluded.availability,
                                              review_num = excluded.review_num;
"""

In [4]:
staging_table_create = """CREATE TABLE IF NOT EXISTS staging_laptop2
                                (   ID SERIAL PRIMARY KEY,
                                    time date NOT NULL,
                                    name varchar NOT NULL,
                                    brand varchar NOT NULL,
                                    sku varchar NOT NULL,
                                    price numeric,
                                    reg_price numeric,
                                    money_saved numeric,
                                    url varchar NOT NULL,
                                    availability varchar,
                                    review_num integer,
                                UNIQUE(sku)
                                );
"""

In [5]:
brand_table_insert = """INSERT INTO dimbrand (name, ticker, exchange_nm)
                            VALUES (%(name)s,
                                    %(ticker)s,
                                    %(exchange_nm)s)
                            ON CONFLICT (name)
                            DO UPDATE SET ticker = excluded.ticker,
                                          exchange_nm = excluded.exchange_nm;
"""
brand_table_insert_from_staging = """INSERT INTO dimbrand (name, ticker, exchange_nm)
                                        SELECT DISTINCT brand, null, null
                                            FROM staging_laptop2
                                    ON CONFLICT 
                                    DO NOTHING
"""

In [6]:
time_table_insert = """INSERT INTO dimtime (time, day, week, month, year, weekday)
                            VALUES (%s, %s, %s, %s, %s, %s)
                       ON CONFLICT (time)
                       DO NOTHING
                    """

In [7]:
laptopinfo_table_insert = """INSERT INTO factlaptopinfo (time, laptop_key, brand_key, price, reg_price, money_saved, availability, review_num)
                                SELECT %(time)s, 
                                       laptop.laptop_key, 
                                       brand.brand_key,
                                       staging.price,
                                       staging.reg_price,
                                       staging.money_saved,
                                       staging.availability,
                                       staging.review_num
                                FROM staging_laptop2 staging
                                JOIN dimlaptop laptop ON (staging.sku = laptop.sku)
                                JOIN dimbrand brand ON (staging.brand = brand.name);
"""

In [8]:
staging_table_delete = """DROP TABLE IF EXISTS staging_laptop2;"""

In [9]:
def is_last_page(soup, curr_page):
    """
    Determine whether the scraped page is the last page.
    If it's the last page, B&H will redirect scraper to page 1.
    
    input:
    soup - beautiful soup object
    curr_page - the current scraping page
    
    return:
    is_last_page - True/False
    """
    # get a list of pages diaplsy on bottom of the page
    listing_pages = soup.find_all('a', attrs={'data-selenium':'listingPagingLink'})
    listing_pages = [int(x.text) for x in listing_pages]
    
    # if current page is the biggest displayed on the page
    if curr_page == max(listing_pages):
        return True
    else:
        return False

In [10]:
def iter_laptop_from_page(soup):
    """
    Retrieve laptop related information from the page.
    - Product Name
    - Product Brand
    - Product SKU
    - Product Price
    - Product URL
    - Product Availability
    - Product # of Reviews
    
    input:
    soup - beautiful soup object
    
    output:
    page_info_dict - dictionary of each product
    """
    # return a list of laptop info from "itemDetail" divs
    products = soup.find_all('div', attrs={'data-selenium':'miniProductPage'})
    
    print(f'Found {len(products)} in total.')
    # go over the product
    for product in products:
        product_dict = {}
        # get current date as time
        product_dict['time'] = np.datetime_as_string(np.datetime64('now'), unit='D')

        # get product brand & name
        ProductBrandName = product.find('span', attrs={'data-selenium':'miniProductPageProductName'}).text
        try:
            product_dict['brand'] = re.findall('(^[a-zA-Z]+ ?[a-zA-Z]*) [0-9]+', ProductBrandName)[0].lower()
            product_dict['name'] = re.findall('[1][0-9\.]+\".*$', ProductBrandName)[0].lower()
        except:
            ProductBrandNameList = ProductBrandName.split()
            product_dict['name'] = ' '.join(ProductBrandNameList[1:])
            product_dict['brand'] = ProductBrandNameList[0]

        # get product sku
        ProductSku = product.find('div', attrs={'data-selenium':'miniProductPageProductSkuInfo'}).text
        product_dict['sku'] = re.findall('B&H # ([0-9A-Z]+) MFR', ProductSku)[0]

        # get product price
        try: 
            ProductPrice = product.find('span', attrs={'data-selenium':'uppedDecimalPrice'}).text
            ProductPrice = int(re.sub('[$,]', '', ProductPrice)) / 100
            product_dict['price'] = ProductPrice
        except:
            product_dict['price'] = None
            print(f"{ProductBrandName} doesn't have a price.")

        # get product regular price
        try:
            ProductRegPrice = product.find('del', attrs={'data-selenium':'strikeThroughPrice'}).text
            ProductRegPrice = re.findall('\$([0-9\.,]+)', ProductRegPrice)[0].replace(',', '')
            product_dict['reg_price'] = float(ProductRegPrice)
        except:
            product_dict['reg_price'] = None
            print(f"{ProductBrandName} doesn't have a reg price.")

        # get money saved
        try:
            ProductSaved = product.find('span', attrs={'data-selenium':'defaultSavingLabel'}).text
            ProductSaved = re.findall('\$([0-9\.,]+)', ProductSaved)[0].replace(',', '')
            product_dict['money_saved'] = float(ProductSaved)
        except:
            product_dict['money_saved'] = None
            print(f"{ProductBrandName} doesn't have money saved info.")

        # get product URL
        product_dict['url'] = product.find('a', attrs={'data-selenium':'miniProductPageProductImgLink'})['href'].lower()

        # get product availability info
        try:
            productAvalability = product.find('span', attrs={'data-selenium':'stockStatus'}).text.lower()
            product_dict['availability'] = productAvalability
        except: 
            product_dict['availability'] = None
            print(f"{ProductBrandName} doesn't have availability info.")

        # get number of reviews
        try:
            reviews_str = product.find('span', attrs={'data-selenium':'miniProductPageProductReviews'}).text.replace(',', '')
            reviews_str = re.findall('([0-9\.]+) Review', reviews_str)[0]
            reviews_int = int(reviews_str)
        except:
            reviews_int = 0

        product_dict['review_num'] = reviews_int

        yield product_dict
        

In [11]:
def iter_laptop_from_site():
    """
    It's a generator function.
    It scrapes data from URL 'https://www.bhphotovideo.com/c/buy/laptops/ci/18818/N/4110474292',
    and return product information.
    
    input:
    page_size - the number of pages in total to search on B&H laptop section
    
    output:
    product_info - a dictionary contains all necessary info for one product  
    """
    # B&H laptop URL
    url = 'https://www.bhphotovideo.com/c/buy/laptops/ci/18818/N/4110474292/pn/'
    # scrape from the first page
    page = 1
    
    while True:
        # make request for each page, get source code and parse with BeautifulSoup
        print(f'Prepare to scrape for page {page}.')
        try:
            print(url+str(page))
            req = Request(url+str(page), headers = {'User-Agent':'Mozilla/5.0'})
            thepage = urlopen(req).read()
            page_soup = BeautifulSoup(thepage, 'html.parser')
        except Exception as e:
            print(f"ERROR ocurred when scraping data for page {page}.")
            print(f"ERROR message: {e}")
            break
              
        # parse data on this page
        product_info = iter_laptop_from_page(page_soup)
            
        yield from product_info
        
        # if last page is reached, exit
        if is_last_page(page_soup, page):
            print("Finished scraping all the data.")
            break
        
        # move to the next page
        page += 1

In [12]:
def process_data(cur, conn):
    """
    - Scrape laptop data
    
    - create staging_laptop table
    - Store scraped data into staging_laptop table
    
    - extract data from staging_laptop into laptop table
    - extract data from staging_laptop into brand table
    
    input: 
    cur - cursor variable
    conn - database connection
    """
    try:
        print("Started to scrape")
        # get all laptop info and save it as a list
        all_laptop_iter = iter_laptop_from_site()

        # create staging table
        cur.execute(staging_table_create)
        print("Created staging_laptop table.")

        # insert into staging table
        psycopg2.extras.execute_batch(cur, staging_table_insert, all_laptop_iter)
        print("Successfully inserted scrapted data into the staging table.")

        # extract data from staging table and insert into dimlaptop table
        cur.execute(laptop_table_insert)
        print("Successfully inserted scrapted data into laptop table")
        
        # extract brand data from staging table and insert into dimlaptop table
        cur.execute(brand_table_insert_from_staging)

        # extract data from ticker_csv and insert into brand table
        df_ticker = pd.read_csv('../data/brand_ticker_info.csv')
        df_ticker = df_ticker.rename(columns={'brand':'name'})
        psycopg2.extras.execute_batch(cur, brand_table_insert, df_ticker.to_dict(orient='records'))
        print("Successfully inserted data into brand table.")

        # extract time info from staging_laptop table
        cur.execute("SELECT DISTINCT time FROM staging_laptop2")
        curr_time = cur.fetchone()[0]

        # convert np.datetime format to pd.timestamp
        time_dt = pd.to_datetime(curr_time)
        time_str = time_dt.strftime("%Y-%m-%d")
        cur.execute(time_table_insert, (curr_time, time_dt.day, 
                                        time_dt.week, time_dt.month, time_dt.year, time_dt.dayofweek))
        print("Successfully inserted data into time table.")

        # insert laptop info into laptop table
        cur.execute(laptopinfo_table_insert, {'time':time_str})
        print("Successfully inserted data into laptopinfo table.")

        # delete staging table
        cur.execute(staging_table_delete)
        print("Deleted staging_laptop table.")
        
        # commit 
        conn.commit()
    except Exception as e:
        print(f'Error happened: {e}')
        conn.rollback()
    

In [13]:
def main():
    """
    - Connect to postgresql database
    
    - get cursor variable
    
    - extract, transform and load data from B&H site
    
    - close database connection
    
    input: None
    return: None
    """
    # connect to database
    print("Started to scrape data for {}".format(np.datetime_as_string(np.datetime64('now'), unit='D')))
    conn = psycopg2.connect(host='localhost', 
                            dbname='bnhlaptop', 
                            password='test', 
                            port=5432, 
                            user='postgres')
    cur = conn.cursor()
    print("Connected to database bnhlaptop.")
    
    # process data
    process_data(cur, conn)

    # close database connection
    conn.close()

In [14]:
main()

Started to scrape data for 2020-08-07
Connected to database bnhlaptop.
Started to scrape
Created staging_laptop table.
Prepare to scrape for page 1.
https://www.bhphotovideo.com/c/buy/laptops/ci/18818/N/4110474292/pn/1
Found 30 in total.
Apple 13.3" MacBook Pro with Retina Display (Mid 2020, Space Gray) doesn't have a reg price.
Apple 13.3" MacBook Pro with Retina Display (Mid 2020, Space Gray) doesn't have money saved info.
Apple 13.3" MacBook Air with Retina Display (Early 2020, Space Gray) doesn't have a reg price.
Apple 13.3" MacBook Air with Retina Display (Early 2020, Space Gray) doesn't have money saved info.
Apple 13.3" MacBook Air with Retina Display (Early 2020, Silver) doesn't have a reg price.
Apple 13.3" MacBook Air with Retina Display (Early 2020, Silver) doesn't have money saved info.
Apple 13.3" MacBook Pro with Retina Display (Mid 2020, Space Gray) doesn't have a reg price.
Apple 13.3" MacBook Pro with Retina Display (Mid 2020, Space Gray) doesn't have money saved info

Found 30 in total.
Acer 15.6" 32GB Chromebook 315 (Pure Silver) doesn't have a reg price.
Acer 15.6" 32GB Chromebook 315 (Pure Silver) doesn't have money saved info.
Apple 13.3" MacBook Air with Retina Display (Early 2020, Silver) doesn't have a reg price.
Apple 13.3" MacBook Air with Retina Display (Early 2020, Silver) doesn't have money saved info.
Apple 13.3" MacBook Air with Retina Display (Early 2020, Space Gray) doesn't have a reg price.
Apple 13.3" MacBook Air with Retina Display (Early 2020, Space Gray) doesn't have money saved info.
Apple 16" MacBook Pro (Late 2019, Space Gray) doesn't have a reg price.
Apple 16" MacBook Pro (Late 2019, Space Gray) doesn't have money saved info.
ASUS 14" VivoBook Flip 14 J401MA Multi-Touch 2-in-1 Laptop doesn't have a reg price.
ASUS 14" VivoBook Flip 14 J401MA Multi-Touch 2-in-1 Laptop doesn't have money saved info.
ASUS 14" ZenBook 14 UX431FA Laptop (Utopia Blue) doesn't have a reg price.
ASUS 14" ZenBook 14 UX431FA Laptop (Utopia Blue) does

Found 30 in total.
Lenovo 15.6" Legion 5 Gaming Laptop doesn't have a reg price.
Lenovo 15.6" Legion 5 Gaming Laptop doesn't have money saved info.
Razer 15.6" Blade 15 Gaming Laptop doesn't have a reg price.
Razer 15.6" Blade 15 Gaming Laptop doesn't have money saved info.
Apple 16" MacBook Pro (Late 2019, Space Gray) doesn't have a reg price.
Apple 16" MacBook Pro (Late 2019, Space Gray) doesn't have money saved info.
Apple 16" MacBook Pro (Late 2019, Silver) doesn't have a reg price.
Apple 16" MacBook Pro (Late 2019, Silver) doesn't have money saved info.
Razer 15.6" Blade 15 Multi-Touch Laptop (2020, Studio Edition) doesn't have a reg price.
Razer 15.6" Blade 15 Multi-Touch Laptop (2020, Studio Edition) doesn't have money saved info.
Lenovo 11.6" 32GB Chromebook 100e doesn't have a reg price.
Lenovo 11.6" 32GB Chromebook 100e doesn't have money saved info.
ASUS 11.6" 32GB Chromebook (Red) doesn't have a reg price.
ASUS 11.6" 32GB Chromebook (Red) doesn't have money saved info.
Leno

Found 30 in total.
ASUS 15.6" Republic of Gamers Zephyrus Duo 15 Gaming Laptop doesn't have a reg price.
ASUS 15.6" Republic of Gamers Zephyrus Duo 15 Gaming Laptop doesn't have money saved info.
Acer 15.6" ConceptD 7 Laptop doesn't have a reg price.
Acer 15.6" ConceptD 7 Laptop doesn't have money saved info.
HP 15.6" ZBook Studio x360 G5 Multi-Touch 2-in-1 Mobile Workstation doesn't have a reg price.
HP 15.6" ZBook Studio x360 G5 Multi-Touch 2-in-1 Mobile Workstation doesn't have money saved info.
Apple 16" MacBook Pro (Late 2019, Space Gray) doesn't have a reg price.
Apple 16" MacBook Pro (Late 2019, Space Gray) doesn't have money saved info.
Apple 16" MacBook Pro (Late 2019, Space Gray) doesn't have a reg price.
Apple 16" MacBook Pro (Late 2019, Space Gray) doesn't have money saved info.
Apple 16" MacBook Pro (Late 2019, Silver) doesn't have a reg price.
Apple 16" MacBook Pro (Late 2019, Silver) doesn't have money saved info.
Apple 16" MacBook Pro (Late 2019, Space Gray) doesn't hav

Found 30 in total.
Samsung 13.3" Galaxy Book S Multi-Touch Laptop (Mercury Gray) doesn't have a reg price.
Samsung 13.3" Galaxy Book S Multi-Touch Laptop (Mercury Gray) doesn't have money saved info.
Lenovo 15.6" ThinkPad E15 Laptop doesn't have a reg price.
Lenovo 15.6" ThinkPad E15 Laptop doesn't have money saved info.
MSI 15.6" GF65 Thin Gaming Laptop doesn't have a reg price.
MSI 15.6" GF65 Thin Gaming Laptop doesn't have money saved info.
Dell 15.6" Latitude 5500 Laptop doesn't have a reg price.
Dell 15.6" Latitude 5500 Laptop doesn't have money saved info.
Apple 13.3" MacBook Air with Retina Display (Early 2020, Silver) doesn't have a reg price.
Apple 13.3" MacBook Air with Retina Display (Early 2020, Silver) doesn't have money saved info.
Lenovo 14" ThinkPad T495 Laptop doesn't have a reg price.
Lenovo 14" ThinkPad T495 Laptop doesn't have money saved info.
Dell 14" Latitude 5400 Laptop doesn't have a reg price.
Dell 14" Latitude 5400 Laptop doesn't have money saved info.
Micros

...

Found 30 in total.
Lenovo 15.6" ThinkPad P1 Laptop Workstation (Gen 2, Black Paint) doesn't have a reg price.
Lenovo 15.6" ThinkPad P1 Laptop Workstation (Gen 2, Black Paint) doesn't have money saved info.
Apple 16" MacBook Pro (Mid 2020, Space Gray) doesn't have a reg price.
Apple 16" MacBook Pro (Mid 2020, Space Gray) doesn't have money saved info.
Apple 16" MacBook Pro (Mid 2020, Silver) doesn't have a reg price.
Apple 16" MacBook Pro (Mid 2020, Silver) doesn't have money saved info.
Apple 13.3" MacBook Pro with Retina Display (Mid 2020, Space Gray) doesn't have a reg price.
Apple 13.3" MacBook Pro with Retina Display (Mid 2020, Space Gray) doesn't have money saved info.
Apple 13.3" MacBook Pro with Retina Display (Mid 2020, Silver) doesn't have a reg price.
Apple 13.3" MacBook Pro with Retina Display (Mid 2020, Silver) doesn't have money saved info.
Apple 16" MacBook Pro (Late 2019, Space Gray) doesn't have a reg price.
Apple 16" MacBook Pro (Late 2019, Space Gray) doesn't have mon

Found 30 in total.
Apple 16" MacBook Pro (Late 2019, Space Gray) doesn't have a reg price.
Apple 16" MacBook Pro (Late 2019, Space Gray) doesn't have money saved info.
Apple 16" MacBook Pro (Late 2019, Space Gray) doesn't have a reg price.
Apple 16" MacBook Pro (Late 2019, Space Gray) doesn't have money saved info.
Apple 16" MacBook Pro (Late 2019, Silver) doesn't have a reg price.
Apple 16" MacBook Pro (Late 2019, Silver) doesn't have money saved info.
Apple 16" MacBook Pro (Late 2019, Silver) doesn't have a reg price.
Apple 16" MacBook Pro (Late 2019, Silver) doesn't have money saved info.
Apple 16" MacBook Pro (Late 2019, Silver) doesn't have a reg price.
Apple 16" MacBook Pro (Late 2019, Silver) doesn't have money saved info.
Apple 16" MacBook Pro (Late 2019, Silver) doesn't have a reg price.
Apple 16" MacBook Pro (Late 2019, Silver) doesn't have money saved info.
Apple 16" MacBook Pro (Late 2019, Space Gray) doesn't have a reg price.
Apple 16" MacBook Pro (Late 2019, Space Gray) d

Found 30 in total.
Apple 16" MacBook Pro (Late 2019, Silver) doesn't have a reg price.
Apple 16" MacBook Pro (Late 2019, Silver) doesn't have money saved info.
ASUS 15.6" ProArt StudioBook Pro W500G5T Mobile Workstation doesn't have a reg price.
ASUS 15.6" ProArt StudioBook Pro W500G5T Mobile Workstation doesn't have money saved info.
Acer 15.6" ConceptD 7 Pro Laptop doesn't have a reg price.
Acer 15.6" ConceptD 7 Pro Laptop doesn't have money saved info.
Lenovo 15.6" ThinkPad P53 Mobile Workstation doesn't have a reg price.
Lenovo 15.6" ThinkPad P53 Mobile Workstation doesn't have money saved info.
Lenovo 17.3" P73 ThinkPad Laptop doesn't have a reg price.
Lenovo 17.3" P73 ThinkPad Laptop doesn't have money saved info.
Panasonic CF-33/ i5-7300U/ 2.6Ghz/ 8GB/ 256SSD/ WF/ Dp/ Win10 Pro/ IR WC/ 8MP Cam/ 12"/ Multi Tch + Digitizer doesn't have a reg price.
Panasonic CF-33/ i5-7300U/ 2.6Ghz/ 8GB/ 256SSD/ WF/ Dp/ Win10 Pro/ IR WC/ 8MP Cam/ 12"/ Multi Tch + Digitizer doesn't have money saved

Found 30 in total.
Apple 16" MacBook Pro (Late 2019, Silver) doesn't have a reg price.
Apple 16" MacBook Pro (Late 2019, Silver) doesn't have money saved info.
Apple 16" MacBook Pro (Late 2019, Space Gray) doesn't have a reg price.
Apple 16" MacBook Pro (Late 2019, Space Gray) doesn't have money saved info.
Apple 16" MacBook Pro (Late 2019, Silver) doesn't have a reg price.
Apple 16" MacBook Pro (Late 2019, Silver) doesn't have money saved info.
Panasonic CF-33 PSS EXL 2/ i7-7600U/ 2.8Ghz/ 16GB/ 512 GB SSD/ WF/ DP/ W10P/ 8MP Cam/ FP/ 12"/ MT + Digitizer doesn't have a reg price.
Panasonic CF-33 PSS EXL 2/ i7-7600U/ 2.8Ghz/ 16GB/ 512 GB SSD/ WF/ DP/ W10P/ 8MP Cam/ FP/ 12"/ MT + Digitizer doesn't have money saved info.
Apple 16" MacBook Pro (Mid 2020, Space Gray) doesn't have a reg price.
Apple 16" MacBook Pro (Mid 2020, Space Gray) doesn't have money saved info.
Apple 16" MacBook Pro (Mid 2020, Silver) doesn't have a reg price.
Apple 16" MacBook Pro (Mid 2020, Silver) doesn't have money

Found 29 in total.
Apple 16" MacBook Pro (Late 2019, Silver) doesn't have a reg price.
Apple 16" MacBook Pro (Late 2019, Silver) doesn't have money saved info.
Apple 16" MacBook Pro (Late 2019, Silver) doesn't have a reg price.
Apple 16" MacBook Pro (Late 2019, Silver) doesn't have money saved info.
Apple 16" MacBook Pro (Mid 2020, Space Gray) doesn't have a reg price.
Apple 16" MacBook Pro (Mid 2020, Space Gray) doesn't have money saved info.
Apple 16" MacBook Pro (Mid 2020, Silver) doesn't have a reg price.
Apple 16" MacBook Pro (Mid 2020, Silver) doesn't have money saved info.
Apple 16" MacBook Pro (Late 2019, Space Gray) doesn't have a reg price.
Apple 16" MacBook Pro (Late 2019, Space Gray) doesn't have money saved info.
Apple 16" MacBook Pro (Late 2019, Space Gray) doesn't have a reg price.
Apple 16" MacBook Pro (Late 2019, Space Gray) doesn't have money saved info.
Apple 16" MacBook Pro (Late 2019, Silver) doesn't have a reg price.
Apple 16" MacBook Pro (Late 2019, Silver) doesn

In [62]:
# B&H laptop URL
url = 'https://www.bhphotovideo.com/c/buy/laptops/ci/18818/N/4110474292/pn/'
# scrape from the first page
page = 17

print(url+str(page))
req = Request(url+str(page), headers = {'User-Agent':'Mozilla/5.0'})
thepage = urlopen(req).read()
page_soup = BeautifulSoup(thepage, 'html.parser')


https://www.bhphotovideo.com/c/buy/laptops/ci/18818/N/4110474292/pn/17


In [63]:
products = page_soup.find_all('div', attrs={'data-selenium':'miniProductPage'})

print(f'Found {len(products)} in total.')
# go over the product
for product in products:

    # get product brand & name
    ProductBrandName = product.find('span', attrs={'data-selenium':'miniProductPageProductName'}).text
    print(ProductBrandName)

Found 30 in total.
HP 14" ProBook 445 G7 Laptop
Lenovo 15.6" ThinkPad E595 Laptop (Black)
Dell 13.3" Latitude 3300 Laptop
Lenovo 15.6" ThinkPad E15 Laptop
Lenovo 15.6" ThinkPad L590 Laptop
HP 15.6" Pavilion 15-cs3010nr Multi-Touch Laptop
Lenovo 14" ThinkPad E495 Laptop (Black)
Lenovo 11.6" 11e ThinkPad Yoga Gen 6 Multi-Touch 2-in-1 Laptop
HP 15.6" ProBook 450 G7 Laptop
HP 13.3" ProBook x360 435 G7 Multi-Touch 2-in-1 Laptop
MSI 14" Modern Series Modern 14 Laptop
HP 15.6" 250 G7 Series Laptop
Dell 14" Latitude 3410 Laptop
Acer Aspire 7 Laptop/ i5-1035G1/ 8Gb/ 512Gb / Windows 10 Home
Acer Aspire 7 Laptop 9G/ i5-9300H/ 8Gb/ 512Gb Ssd/ Windows 10 Home
ASUS UX425-EB51/ i5-1035G1/ 8GB/ 512GB SSD/ UHD/ Windows 10 Home/ 14"
MSI 15.6" Creator Series Modern 15 Laptop
HP 14" ProBook 440 G7 Multi-Touch Laptop
MSI 15.6" Bravo 15 Gaming Laptop
HP 14" ProBook 440 G6 Notebook
Lenovo 13.3" ThinkPad L13 Yoga Multi-Touch 2-in-1 Laptop
Lenovo 15.6" ThinkPad E15 Laptop (Silver)
Lenovo 14" ThinkPad E495 Lapt